### 금융상품 갱신 여부 예측하는 ANN

Churn_Modelling.csv 파일을 보면, 고객 정보와 해당 고객이 금융상품을 갱신했는지 안했는지의 여부에 대한 데이터가 있다.

이 데이터를 가지고 갱신여부를 예측하는 딥러닝을 구성하시오.

# Neural Networks and Deep Learning

In [107]:
# Importing the libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
%matplotlib inline

In [108]:
# 1. csv 파일을, 구글이 제공한 컴퓨터에 업로드 하여 사용하는 방법

In [109]:
# df = pd.read_csv('Churn_Modelling.csv')

In [110]:
# 2. 내 구글 드라이브에 csv파일을 갖다놓고, 코랩을 연결시키는 방법

In [111]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [112]:
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/ML2/data/Churn_Modelling.csv')

In [113]:
df

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,9996,15606229,Obijiaku,771,France,Male,39,5,0.00,2,1,0,96270.64,0
9996,9997,15569892,Johnstone,516,France,Male,35,10,57369.61,1,1,1,101699.77,0
9997,9998,15584532,Liu,709,France,Female,36,7,0.00,1,0,1,42085.58,1
9998,9999,15682355,Sabbatini,772,Germany,Male,42,3,75075.31,2,1,0,92888.52,1


In [114]:
df.isna().sum()

RowNumber          0
CustomerId         0
Surname            0
CreditScore        0
Geography          0
Gender             0
Age                0
Tenure             0
Balance            0
NumOfProducts      0
HasCrCard          0
IsActiveMember     0
EstimatedSalary    0
Exited             0
dtype: int64

In [115]:
df.head(2)

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0


In [116]:
y = df[ 'Exited']

In [117]:
df.head(2)

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0


In [118]:
X = df.loc[ : , 'CreditScore' :  'EstimatedSalary' ]

In [119]:
X

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,619,France,Female,42,2,0.00,1,1,1,101348.88
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58
2,502,France,Female,42,8,159660.80,3,1,0,113931.57
3,699,France,Female,39,1,0.00,2,0,0,93826.63
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10
...,...,...,...,...,...,...,...,...,...,...
9995,771,France,Male,39,5,0.00,2,1,0,96270.64
9996,516,France,Male,35,10,57369.61,1,1,1,101699.77
9997,709,France,Female,36,7,0.00,1,0,1,42085.58
9998,772,Germany,Male,42,3,75075.31,2,1,0,92888.52


In [120]:
# prompt: Geography 컬럼은 몇개의 데이터로 구성되어있나

unique_geography = df['Geography'].unique()
print(len(unique_geography))


3


In [121]:
# prompt: Geography 컬럼에 nunique 해줘

df['Geography'].nunique()



3

In [122]:
X['Gender'].nunique()

2

In [123]:
# prompt: gender 컬럼을 레이블 인코딩 해줘

from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
X['Gender'] = label_encoder.fit_transform(X['Gender'])
X.head(2)


,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,619,France,0,42,2,0.00,1,1,1,101348.88
1,608,Spain,0,41,1,83807.86,1,0,1,112542.58


In [124]:
sorted( df['Geography'].unique() )

['France', 'Germany', 'Spain']

In [125]:
df

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,9996,15606229,Obijiaku,771,France,Male,39,5,0.00,2,1,0,96270.64,0
9996,9997,15569892,Johnstone,516,France,Male,35,10,57369.61,1,1,1,101699.77,0
9997,9998,15584532,Liu,709,France,Female,36,7,0.00,1,0,1,42085.58,1
9998,9999,15682355,Sabbatini,772,Germany,Male,42,3,75075.31,2,1,0,92888.52,1


In [126]:
# prompt: geograpy 컬럼을 columtransformer 로 원핫 인코딩 해줘

from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

ct = ColumnTransformer(
    [('encoding', OneHotEncoder(), [1])],   # The column numbers to be encoded
    remainder='passthrough'                         # Leave other columns as is
)

X = ct.fit_transform(X)


In [127]:
X

array([[1.0000000e+00, 0.0000000e+00, 0.0000000e+00, ..., 1.0000000e+00,
        1.0000000e+00, 1.0134888e+05],
       [0.0000000e+00, 0.0000000e+00, 1.0000000e+00, ..., 0.0000000e+00,
        1.0000000e+00, 1.1254258e+05],
       [1.0000000e+00, 0.0000000e+00, 0.0000000e+00, ..., 1.0000000e+00,
        0.0000000e+00, 1.1393157e+05],
       ...,
       [1.0000000e+00, 0.0000000e+00, 0.0000000e+00, ..., 0.0000000e+00,
        1.0000000e+00, 4.2085580e+04],
       [0.0000000e+00, 1.0000000e+00, 0.0000000e+00, ..., 1.0000000e+00,
        0.0000000e+00, 9.2888520e+04],
       [1.0000000e+00, 0.0000000e+00, 0.0000000e+00, ..., 1.0000000e+00,
        0.0000000e+00, 3.8190780e+04]])

In [128]:
# Dummy Variable Trap 을 적용하자.

# 딥러닝에서는 컬럼하나가 연산에 아주 크게 작용하므로,

# 원핫 인코딩한 결과에서, 가장 왼쪽의 컬럼은 삭제해도
# 데이터를 표현하는 데는 아무 문제 없다.

# France, Germerny, Spain 3개 컬럼으로 원핫 인코딩 되는데
#    1        0       0
#    0        1       0
#    0        0       1

# 맨 왼쪽 France 컬럼을 삭제해도,
#  0   0   => France
#  1   0   => Germeny
#  0   1   => Spain

In [129]:
pd.DataFrame(X).drop(0, axis=1).values

array([[0.0000000e+00, 0.0000000e+00, 6.1900000e+02, ..., 1.0000000e+00,
        1.0000000e+00, 1.0134888e+05],
       [0.0000000e+00, 1.0000000e+00, 6.0800000e+02, ..., 0.0000000e+00,
        1.0000000e+00, 1.1254258e+05],
       [0.0000000e+00, 0.0000000e+00, 5.0200000e+02, ..., 1.0000000e+00,
        0.0000000e+00, 1.1393157e+05],
       ...,
       [0.0000000e+00, 0.0000000e+00, 7.0900000e+02, ..., 0.0000000e+00,
        1.0000000e+00, 4.2085580e+04],
       [1.0000000e+00, 0.0000000e+00, 7.7200000e+02, ..., 1.0000000e+00,
        0.0000000e+00, 9.2888520e+04],
       [0.0000000e+00, 0.0000000e+00, 7.9200000e+02, ..., 1.0000000e+00,
        0.0000000e+00, 3.8190780e+04]])

In [130]:
# prompt: X 의 첫번째 컬럼을 삭제해줘

X = pd.DataFrame(X).drop(0, axis=1).values


In [131]:
X

array([[0.0000000e+00, 0.0000000e+00, 6.1900000e+02, ..., 1.0000000e+00,
        1.0000000e+00, 1.0134888e+05],
       [0.0000000e+00, 1.0000000e+00, 6.0800000e+02, ..., 0.0000000e+00,
        1.0000000e+00, 1.1254258e+05],
       [0.0000000e+00, 0.0000000e+00, 5.0200000e+02, ..., 1.0000000e+00,
        0.0000000e+00, 1.1393157e+05],
       ...,
       [0.0000000e+00, 0.0000000e+00, 7.0900000e+02, ..., 0.0000000e+00,
        1.0000000e+00, 4.2085580e+04],
       [1.0000000e+00, 0.0000000e+00, 7.7200000e+02, ..., 1.0000000e+00,
        0.0000000e+00, 9.2888520e+04],
       [0.0000000e+00, 0.0000000e+00, 7.9200000e+02, ..., 1.0000000e+00,
        0.0000000e+00, 3.8190780e+04]])

In [132]:
# prompt: X 를 스탠다드 스케일러로 피처 스케일링 해줘

from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X = sc.fit_transform(X)


In [133]:
y

0       1
1       0
2       1
3       0
4       0
       ..
9995    0
9996    0
9997    1
9998    1
9999    0
Name: Exited, Length: 10000, dtype: int64

In [134]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [135]:
# Part 2 - Now let's make the ANN!

import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense

In [136]:
X.shape

(10000, 11)

In [137]:
X[ 0 , ]

array([-0.57873591, -0.57380915, -0.32622142, -1.09598752,  0.29351742,
       -1.04175968, -1.22584767, -0.91158349,  0.64609167,  0.97024255,
        0.02188649])

In [138]:
# 깡통 인공지능
model = Sequential()

In [139]:
model.add(  Dense(units = 8, activation='relu', input_shape= (11,)  )   )

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:86: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [140]:
model.add(  Dense(6, 'relu')  )

In [141]:
model.add(  Dense(1, 'sigmoid') )

In [142]:
12*8

96

In [143]:
model.summary()

Model: "sequential_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense_15 (Dense)                     │ (None, 8)                   │              96 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_16 (Dense)                     │ (None, 6)                   │              54 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_17 (Dense)                     │ (None, 1)                   │               7 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 157 (628.00 B)

 Trainable params: 157 (628.00 B)

 Non-trainable params: 0 (0.00 B)

In [144]:
# 모델링이 끝나면, 컴파일(Compile)을 해야 한다.
# 컴파일이란,
#     옵티마이저(optimizer) 셋팅
#     로스펑션(loss function 오차함수,손실함수) 셋팅
#     검증(평가)방법 셋팅

In [145]:

# 2개로 분류하는 문제의 loss는 'binary_crossentropy' 로 설정한다.

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'] )

In [146]:
# 컴파일이 끝나면, 학습을 한다.

In [147]:
model.fit( X_train, y_train, batch_size = 10, epochs = 20  )

Epoch 1/20
800/800 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.6833 - loss: 0.6067
Epoch 2/20
800/800 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.8210 - loss: 0.4225
Epoch 3/20
800/800 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.8227 - loss: 0.4275
Epoch 4/20
800/800 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.8280 - loss: 0.4077
Epoch 5/20
800/800 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.8279 - loss: 0.4020
Epoch 6/20
800/800 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.8436 - loss: 0.3812
Epoch 7/20
800/800 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.8551 - loss: 0.3552
Epoch 8/20
800/800 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.8582 - loss: 0.3481
Epoch 9/20
800/800 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.8573 - loss: 0.3526
Epoch 10/20
800/800 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8618 - loss: 0.3438
Epoch 11/20
800/800 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.8547 - loss: 0.3468
Epoch 12/20
800/800 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step

In [148]:
# 학습이 끝나면, 평가를한다. => X_test

In [149]:
model.evaluate( X_test, y_test )

63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8560 - loss: 0.3255


[0.33433398604393005, 0.8569999933242798]

In [150]:
# 컨퓨전 매트릭스로 확인!!

In [151]:
from sklearn.metrics import confusion_matrix, accuracy_score

In [152]:
y_pred = model.predict(X_test)

63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step


In [153]:
y_pred

array([[0.28815103],
       [0.29438376],
       [0.17301226],
       ...,
       [0.12265741],
       [0.18287136],
       [0.30215517]], dtype=float32)

In [154]:
y_pred = (y_pred > 0.5).astype(int)

In [155]:
y_pred

array([[0],
       [0],
       [0],
       ...,
       [0],
       [0],
       [0]])

In [156]:
y_test

9394    0
898     1
2398    0
5906    0
2343    0
       ..
1037    0
2899    0
9549    0
2740    0
6690    0
Name: Exited, Length: 2000, dtype: int64

In [157]:
cm = confusion_matrix(y_test, y_pred)

In [158]:
cm

array([[1509,   86],
       [ 200,  205]])

In [159]:
(1518+207) / cm.sum()

0.8625

In [160]:
accuracy_score(y_test, y_pred)

0.857

In [161]:
model.evaluate(X_test, y_test)

63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8560 - loss: 0.3255


[0.33433398604393005, 0.8569999933242798]

In [162]:
model.layers

[<Dense name=dense_15, built=True>,
 <Dense name=dense_16, built=True>,
 <Dense name=dense_17, built=True>]

In [163]:
model.layers[0].get_weights()

[array([[-0.19334322, -0.30010515,  0.1894567 ,  0.00541613,  0.24264194,
         -0.10095752,  0.3124565 , -0.7886313 ],
        [ 0.04181432,  0.14826147,  0.0276606 ,  0.05430221,  0.08517415,
         -0.21622281,  0.1748108 , -0.10168644],
        [-0.11320402, -0.0212447 , -0.01755705,  0.04988526,  0.05630485,
          0.07748552,  0.05127348, -0.02616777],
        [ 0.11769666,  0.2244288 ,  0.00254185,  0.12126834, -0.05842486,
         -0.1336557 ,  0.19087979,  0.28913695],
        [ 0.8982742 , -0.13086028,  0.29775664,  0.37137687,  1.037511  ,
         -0.8922556 , -0.70226115, -0.57883734],
        [ 0.02592502, -0.29222912, -0.04693931,  0.17102078,  0.3168543 ,
          0.12841798, -0.16853376,  0.06355106],
        [-0.5620727 , -0.12082772, -0.292097  , -0.08386695,  0.46871963,
          0.7669578 , -0.36105788, -0.27926546],
        [-0.31829628,  1.2636247 , -1.3210058 , -0.28619218,  0.39320076,
         -0.42876372, -0.11396457, -0.24381618],
        [ 0.0160

In [164]:
# 인풋레이어에서 히든레이어로 연결하는 선 (방정식의 계수)
#  11 * 8
model.layers[0].get_weights()[0]

array([[-0.19334322, -0.30010515,  0.1894567 ,  0.00541613,  0.24264194,
        -0.10095752,  0.3124565 , -0.7886313 ],
       [ 0.04181432,  0.14826147,  0.0276606 ,  0.05430221,  0.08517415,
        -0.21622281,  0.1748108 , -0.10168644],
       [-0.11320402, -0.0212447 , -0.01755705,  0.04988526,  0.05630485,
         0.07748552,  0.05127348, -0.02616777],
       [ 0.11769666,  0.2244288 ,  0.00254185,  0.12126834, -0.05842486,
        -0.1336557 ,  0.19087979,  0.28913695],
       [ 0.8982742 , -0.13086028,  0.29775664,  0.37137687,  1.037511  ,
        -0.8922556 , -0.70226115, -0.57883734],
       [ 0.02592502, -0.29222912, -0.04693931,  0.17102078,  0.3168543 ,
         0.12841798, -0.16853376,  0.06355106],
       [-0.5620727 , -0.12082772, -0.292097  , -0.08386695,  0.46871963,
         0.7669578 , -0.36105788, -0.27926546],
       [-0.31829628,  1.2636247 , -1.3210058 , -0.28619218,  0.39320076,
        -0.42876372, -0.11396457, -0.24381618],
       [ 0.01602831,  0.04690674

In [165]:
# 상수항과 히든레이어와 연결된 선 ( 상수항 )
model.layers[0].get_weights()[1]

array([-0.67672706,  0.18420026,  0.22155432,  0.041793  , -0.2057961 ,
        0.31319872,  0.77599096,  0.8893578 ], dtype=float32)

In [166]:
model.layers[1].get_weights()[0]

array([[ 0.60574186, -0.6702307 ,  0.66370654, -0.35780874,  0.30769852,
         0.7897881 ],
       [-0.7779923 ,  0.86059374, -0.98324025,  0.03356318, -0.32815024,
        -0.12262266],
       [-0.48117414,  0.82294774,  0.24023014, -0.7459151 , -0.5325477 ,
        -0.6516738 ],
       [-0.34231153, -0.5780502 , -0.1506134 ,  0.43167207,  0.19354616,
        -0.11991722],
       [-0.49341512,  0.48783135, -0.74445397, -0.510096  ,  0.46253514,
         0.3350615 ],
       [-0.4321217 , -0.58054715, -0.5308292 , -0.20672175,  0.2089783 ,
        -0.36231974],
       [ 0.30390137, -0.4088516 ,  0.23586752,  0.33449227,  0.27048752,
         0.8276174 ],
       [-0.6604142 , -0.5596173 ,  0.3204547 ,  0.51761067,  0.40985122,
        -0.3479229 ]], dtype=float32)

In [167]:
model.layers[1].get_weights()[1]

array([-0.2537725 ,  0.05149407, -0.16766895,  0.8451157 ,  0.44897115,
        0.38684613], dtype=float32)

In [168]:
model.layers[2].get_weights()

[array([[ 0.4617855 ],
        [ 1.5279692 ],
        [ 0.9833732 ],
        [-1.1989908 ],
        [-0.48995617],
        [-0.5280173 ]], dtype=float32),
 array([-0.3095317], dtype=float32)]

## 다음 신규 데이터를 통해 분류해 보자

- Geography: France
- Credit Score: 600
- Gender: Male
- Age: 40
- Tenure: 3
- Balance: 60000
- Number of Products: 2
- Has Credit Card: Yes
- Is Active Member: Yes
- Estimated Salary: 50000

In [169]:
df.head(2)

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0


In [170]:
new_data = [[600, 'France', 'Male', 40, 3, 60000, 2, 1, 1, 50000 ]]

In [171]:
df_new_data = pd.DataFrame(new_data)

In [172]:
df_new_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 10 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   0       1 non-null      int64 
 1   1       1 non-null      object
 2   2       1 non-null      object
 3   3       1 non-null      int64 
 4   4       1 non-null      int64 
 5   5       1 non-null      int64 
 6   6       1 non-null      int64 
 7   7       1 non-null      int64 
 8   8       1 non-null      int64 
 9   9       1 non-null      int64 
dtypes: int64(8), object(2)
memory usage: 208.0+ bytes


In [173]:
df_new_data

,0,1,2,3,4,5,6,7,8,9
0,600,France,Male,40,3,60000,2,1,1,50000


In [174]:
df.columns

Index(['RowNumber', 'CustomerId', 'Surname', 'CreditScore', 'Geography',
       'Gender', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'HasCrCard',
       'IsActiveMember', 'EstimatedSalary', 'Exited'],
      dtype='object')

In [175]:
df_new_data.columns = ['CreditScore', 'Geography',
       'Gender', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'HasCrCard',
       'IsActiveMember', 'EstimatedSalary']

In [176]:
df_new_data

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,Male,40,3,60000,2,1,1,50000


In [177]:
df_new_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   CreditScore      1 non-null      int64 
 1   Geography        1 non-null      object
 2   Gender           1 non-null      object
 3   Age              1 non-null      int64 
 4   Tenure           1 non-null      int64 
 5   Balance          1 non-null      int64 
 6   NumOfProducts    1 non-null      int64 
 7   HasCrCard        1 non-null      int64 
 8   IsActiveMember   1 non-null      int64 
 9   EstimatedSalary  1 non-null      int64 
dtypes: int64(8), object(2)
memory usage: 208.0+ bytes


In [178]:
df_new_data['Gender'] = label_encoder.transform(df_new_data['Gender'])

In [179]:
df_new_data = ct.transform(df_new_data)

In [180]:
new_data = pd.DataFrame(df_new_data).drop(0, axis =1).values

In [181]:
new_data = sc.transform(new_data)

In [182]:
y_pred = model.predict(new_data)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step


In [183]:
(y_pred > 0.5).astype(int)

array([[0]])

In [184]:
new_data2 = [{'CreditScore': 600, 'Geography': 'France', 'Gender': 'Male', 'Age':40, 'Tenure': 3, 'Balance': 60000, 'NumOfProducts': 2, 'HasCrCard': 1, 'IsActiveMember': 1, 'EstimatedSalary':50000 }]

In [185]:
df_new_data2 = pd.DataFrame(new_data2)

In [186]:
df_new_data2['Gender'] = label_encoder.transform(df_new_data2['Gender'])

In [187]:
df_new_data2 = ct.transform(df_new_data2)

In [188]:
new_data2 = pd.DataFrame(df_new_data2).drop(0, axis =1).values

In [189]:
new_data2 = sc.transform(new_data2)

In [190]:
new_data2

array([[-0.57873591, -0.57380915, -0.52281016,  0.91241915,  0.10281024,
        -0.69598177, -0.26422114,  0.80773656,  0.64609167,  0.97024255,
        -0.87101922]])

In [191]:
y_pred = model.predict(new_data2)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step


In [192]:
(y_pred > 0.5).astype(int)

array([[0]])

### 용어 정리

epoch

- 한 번의 epoch는 신경망에서 전체 데이터 셋에 대해 forward pass/backward pass 과정을 거친 것을 말함. 즉, 전체 데이터 셋에 대해 한 번 학습을 완료한 상태


batch_size

메모리의 한계와 속도 저하 때문에 대부분의 경우에는 한 번의 epoch에서 모든 데이터를 한꺼번에 집어넣을 수는 없습니다. 그래서 데이터를 나누어서 주게 되는데 이때 몇 번 나누어서 주는가를 iteration, 각 iteration마다 주는 데이터 사이즈를 batch size라고 합니다.

출처: https://www.slideshare.net/w0ong/ss-82372826

### GridSearch 를 이용한, 최적의 하이퍼 파라미터 찾기

In [193]:
! pip install scikeras

In [194]:
# Tuning the ANN
from scikeras.wrappers import KerasClassifier
from sklearn.model_selection import GridSearchCV
from keras.models import Sequential
from keras.layers import Dense


In [195]:
def build_model(optimizer = 'adam') :
  model = Sequential()
  model.add(Dense(8, 'relu', input_shape = (11,)))
  model.add(Dense(6, 'relu'))
  model.add(Dense(1, 'sigmoid'))
  model.compile(optimizer = optimizer, loss = 'binary_crossentropy', metrics = ['accuracy'])
  return model


In [196]:
model = KerasClassifier(build_fn = build_model)

In [198]:
my_param = {'batch_size' : [10, 20], 'epochs' : [10, 20, 30], 'optimizer':['adam', 'rmsprop']}

In [199]:
grid = GridSearchCV(estimator=model, param_grid= my_param, scoring='accuracy')

In [200]:
grid.fit(X_train, y_train)

Epoch 1/10


/usr/local/lib/python3.10/dist-packages/scikeras/wrappers.py:925: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:86: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


640/640 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.6075 - loss: 0.6357
Epoch 2/10
640/640 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.8058 - loss: 0.4424
Epoch 3/10
640/640 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.8180 - loss: 0.4176
Epoch 4/10
640/640 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.8245 - loss: 0.4029
Epoch 5/10
640/640 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.8198 - loss: 0.4159
Epoch 6/10
640/640 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.8368 - loss: 0.3888
Epoch 7/10
640/640 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.8269 - loss: 0.4045
Epoch 8/10
640/640 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.8353 - loss: 0.3844
Epoch 9/10
640/640 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.8362 - loss: 0.3751
Epoch 10/10
640/640 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.8467 - loss: 0.3729
160/160 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
Epoch 1/10


/usr/local/lib/python3.10/dist-packages/scikeras/wrappers.py:925: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:86: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


640/640 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.7060 - loss: 0.5639
Epoch 2/10
640/640 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.8129 - loss: 0.4305
Epoch 3/10
640/640 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.8389 - loss: 0.3934
Epoch 4/10
640/640 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.8294 - loss: 0.4008
Epoch 5/10
640/640 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.8423 - loss: 0.3814
Epoch 6/10
640/640 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.8423 - loss: 0.3760
Epoch 7/10
640/640 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.8485 - loss: 0.3613
Epoch 8/10
640/640 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.8492 - loss: 0.3676
Epoch 9/10
640/640 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.8603 - loss: 0.3465
Epoch 10/10
640/640 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.8604 - loss: 0.3387
160/160 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
Epoch 1/10


...

/usr/local/lib/python3.10/dist-packages/scikeras/wrappers.py:925: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:86: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


320/320 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.7113 - loss: 0.6341
Epoch 2/30
320/320 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.8071 - loss: 0.4806
Epoch 3/30
320/320 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.8275 - loss: 0.4199
Epoch 4/30
320/320 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8267 - loss: 0.4004
Epoch 5/30
320/320 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.8401 - loss: 0.3747
Epoch 6/30
320/320 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8496 - loss: 0.3611
Epoch 7/30
320/320 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.8624 - loss: 0.3425
Epoch 8/30
320/320 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8484 - loss: 0.3557
Epoch 9/30
320/320 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8604 - loss: 0.3413
Epoch 10/30
320/320 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8518 - loss: 0.3473
Epoch 11/30
320/320 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8651 - loss: 0.3343
Epoch 12/30
320/320 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy

/usr/local/lib/python3.10/dist-packages/scikeras/wrappers.py:925: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:86: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


800/800 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.7578 - loss: 0.5544
Epoch 2/30
800/800 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.7981 - loss: 0.4385
Epoch 3/30
800/800 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.8242 - loss: 0.4080
Epoch 4/30
800/800 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.8271 - loss: 0.4002
Epoch 5/30
800/800 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.8481 - loss: 0.3685
Epoch 6/30
800/800 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.8391 - loss: 0.3842
Epoch 7/30
800/800 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.8566 - loss: 0.3587
Epoch 8/30
800/800 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.8543 - loss: 0.3568
Epoch 9/30
800/800 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.8616 - loss: 0.3491
Epoch 10/30
800/800 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.8621 - loss: 0.3437
Epoch 11/30
800/800 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8633 - loss: 0.3467
Epoch 12/30
800/800 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy

GridSearchCV(estimator=KerasClassifier(build_fn=<function build_model at 0x7cafbed67130>),
             param_grid={'batch_size': [10, 20], 'epochs': [10, 20, 30],
                         'optimizer': ['adam', 'rmsprop']},
             scoring='accuracy')